# jupyter notbook Gui functions
Here we will defin the functions to query a user on the mongodb database

In [1]:
from pymongo import MongoClient

endpoint = '10.0.10.5:27017'
database = 'supercias'
collection = 'consulta_personal'

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
collection = db[collection]

#find one
entry = collection.find_one()

entry = entry['nombre']

print(entry)



PERALTA CARDENAS RIGOBERTO NECTARIO


In [2]:
import ipywidgets as widgets
from IPython.display import display

def Tab_widget(widget_list=[widgets.Text(description='Nombre:')], title_list=('Datos Personales',)):
    # if title_list is a list make into a tuple
    if type(title_list) == list:
        title_list = tuple(title_list)
    tab = widgets.Tab()
    tab.children = widget_list
    tab.titles = title_list
    return tab

def Accordion_widget(widget_list=[widgets.Text(description='Nombre:')], title_list=('Datos Personales',)):
    accordion = widgets.Accordion()
    accordion.children = widget_list
    accordion.titles = title_list
    return accordion

def Label_widget(text):
    label = widgets.Label(value=text)
    return label

label = Label_widget('workig version')
label2 = Label_widget('workig version')
accordion = Accordion_widget([label, label2], ('Datos Personales', 'Datos Laborales'))
accordion2 = Accordion_widget([label, label2], ('Datos Personales', 'Datos Laborales'))
tab = Tab_widget([accordion, accordion2], ('accordion1', 'accordion2'))
display(tab)

## Searchbar

In [3]:
def query_person(nombre, id):
    # query the person
    query= collection.find({
        'nombre': { '$regex': '.*'+nombre+'.*', '$options': 'i' },
        'cedula': { '$regex': '.*'+id+'.*', '$options': 'i' }
        })

    # print how many files were found
    # this will exahust the cursor
    print('found:', len(list(query)))

    # reset the cursor 
    query = collection.find({
        'nombre': { '$regex': '.*'+nombre+'.*', '$options': 'i' },
        'cedula': { '$regex': '.*'+id+'.*', '$options': 'i' }
        })

    results = []
    # make a dataframe with the results
    for record in query:
        results.append(record)
        print(record['nombre'])
        print(record['cedula'])

        if(record['administrador_actual']):
            print('administrador_actual:')
            for admin in record['administrador_actual']:
                print(admin)
        if(record['accionista_actual']):
            print('accionista_actual:')
            for accionista in record['accionista_actual']:
                print(accionista)

        if(record['administradores_anteriores']):
            print('administradores_anteriores:')
            for admin in record['administradores_anteriores']:
                print(admin)

        if(record['accionista_extrajeras']):
            print('accionista_extrajeras:')
            for accionista in record['accionista_extrajeras']:
                print(accionista)
        
        print('-------------------')
    return results

# formated with all caps
nombre = ''
id = '0918588328'
query = query_person(nombre, id)



found: 1
CHIANG GUERRERO LUIS DANIEL
0918588328
administrador_actual:
{'Expediente': '313553', 'Nombre': 'CHIANG S.A.', 'Ruc': '0993190063001', 'Nacionalidad': 'ECUADOR', 'Cargo': 'GERENTE GENERAL', 'Fecha Nombramiento': '2020-07-14', 'Fecha Termino': '', 'Periodo': '5', 'Fecha Registro Mercantil': '2020-07-23', 'Artículo': '6', 'N° Registro Mercantil': '8605', 'RL/ADM': 'RL'}
accionista_actual:
{'Expediente': '300731', 'Nombre': 'INELECOMP S.A.', 'Ruc': '0992902612001', 'Capital Invertido': '400.0', 'Capital Total Cía.': '800', 'Valor Nominal': '1.0', 'Situación Legal': 'CANC. DE INSCRIPCION ANOTADA EN RM', 'Posesión Efectiva': '31355'}
{'Expediente': '313553', 'Nombre': 'CHIANG S.A.', 'Ruc': '0993190063001', 'Capital Invertido': '792.0', 'Capital Total Cía.': '800', 'Valor Nominal': '1.0', 'Situación Legal': 'ACTIVA', 'Posesión Efectiva': ''}
administradores_anteriores:
{'Expediente': '313553', 'Nombre': 'CHIANG S.A.', 'Ruc': '0993190063001', 'Nacionalidad': 'ECUADOR', 'Cargo': 'GERE

In [40]:
import ipywidgets as widgets
from IPython.display import display
import json

def Tab_widget(widget_list=[widgets.Text(description='Nombre:')], title_list=('Datos Personales',)):
    # make into a tuple
    if type(title_list) == list:
        title_list = tuple(title_list)
    tab = widgets.Tab()
    tab.children = widget_list
    tab.titles = title_list
    return tab

def Accordion_widget(widget_list=[widgets.Text(description='Nombre:')], title_list=('Datos Personales',)):
    # make into a tuple
    if type(title_list) == list:
        title_list = tuple(title_list)
    accordion = widgets.Accordion()
    accordion.children = widget_list
    accordion.titles = title_list
    return accordion

def Label_widget(text):
    label = widgets.Label(value=text)
    return label

def format_output(output):
    # print key in the dictionary make a label
    keys_level_1 = list(output.keys())
    # if there is an _id key remove it
    if '_id' in keys_level_1:
        keys_level_1.remove('_id')
    # make into a 
    widgets_level_2 = []
    for key in keys_level_1:
        if type(output[key]) == str:
            # make a widget string
            widgets_level_2.append(Label_widget(key))
        else: # make a accordion widget
            if type(output[key]) == list:
                # convert list to dic
                output[key] = {i: value for i, value in enumerate(output[key])}
            keys_level_2 = [str(key) for key in list(output[key].keys())]
            # stringfy the dictionary
            labels = [Label_widget(json.dumps(output[key][key_l2],indent=4)) for key_l2 in output[key].keys()]
            accordion = Accordion_widget(labels, keys_level_2)
            widgets_level_2.append(accordion)
    tab = Tab_widget(widgets_level_2, keys_level_1)
    # now for each key inside the

    display(tab)
    # make each key into a tab widget


# get the first result
record = query[0]
format_output(record)

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def SearchBar(search_function):
    # Create a text input widget
    cedula_input = widgets.Text(
        value='',
        placeholder='cedula',
        description='Cedula:',
        disabled=False
    )
    # Create a text input widget
    nombre_input = widgets.Text(
        value='',
        placeholder='nombre',
        description='Nombre:',
        disabled=False
    )
    # Create a button widget
    button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='',
        tooltip='Click me',
        icon='check'
    )
    # Function to handle button click
    def on_button_click(b):
        # Clear previous output
        clear_output(wait=True)
        
        # Display the input widgets and button again
        display(input_and_button)
        # retrieve the values from the input widgets
        cedula = cedula_input.value
        nombre = nombre_input.value
        # query the person
        search_function(nombre, cedula)


    # Attach the function to the button click event
    button.on_click(on_button_click)

    # Display text input and button together using HBox
    input_and_button = widgets.HBox([cedula_input, nombre_input, button])
    display(input_and_button)

In [6]:
SearchBar(query_person)

found: 1
TOPIC VERDUGA GORAN ANDJELKO
0916576796
-------------------
